# Download the datasets and unzip

Download the kaggle dataset of cats and dogs images

In [1]:
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
!unzip -q kagglecatsanddogs_3367a.zip 
print ('dataset downloaded')

--2021-06-24 04:02:42--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.215.184.109, 2600:1408:9000:887::e59, 2600:1408:9000:89d::e59
Connecting to download.microsoft.com (download.microsoft.com)|23.215.184.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_3367a.zip’

kagglecatsanddogs_3 100%[===================>] 786.68M   187MB/s    in 4.3s    

2021-06-24 04:02:46 (184 MB/s) - ‘kagglecatsanddogs_3367a.zip’ saved [824894548/824894548]

dataset downloaded


# Create training data

In [2]:
#import all libraries
import os, random, cv2, numpy
print('import done')

import done


In [3]:
categories=['Dog','Cat'] #folder and category names
training_data=[] # empty list that we'll use to store the final data set
img_size=70
print('variables initialized')

variables initialized


In [4]:
print('downsizing dogs and cats')
for i in range(len(categories)):
  print('category is',categories[i] )
  
  for img in os.listdir(f'PetImages/{categories[i]}'): #iterate through images in a particular folder
    #using try except to escape the corrupt images
    try:
        im_array=cv2.imread(f'PetImages/{categories[i]}/{img}',cv2.IMREAD_GRAYSCALE) # load the image in memory and load it as a grayscale(1 channel)
        new_array= cv2.resize(im_array,(img_size,img_size)) #downsize the image to 70x70 size
    except:
        print(f'{img} was skipped')

    training_data.append([new_array, i]) #add the processed image and label to the dataset

#shuffle the dataset to avoid having dogs and cats separately
random.shuffle(training_data)
print("database import complete")
print (f'{len(training_data)} images added')

downsizing dogs and cats
category is Dog
7969.jpg was skipped
7459.jpg was skipped
2688.jpg was skipped
2877.jpg was skipped
11853.jpg was skipped
6238.jpg was skipped
10158.jpg was skipped
10797.jpg was skipped
11675.jpg was skipped
1866.jpg was skipped
2384.jpg was skipped
10401.jpg was skipped
7369.jpg was skipped
10747.jpg was skipped
8730.jpg was skipped
5604.jpg was skipped
4367.jpg was skipped
3136.jpg was skipped
1308.jpg was skipped
6059.jpg was skipped
Thumbs.db was skipped
5736.jpg was skipped
11702.jpg was skipped
11410.jpg was skipped
6718.jpg was skipped
3288.jpg was skipped
9188.jpg was skipped
7112.jpg was skipped
11849.jpg was skipped
3588.jpg was skipped
7133.jpg was skipped
category is Cat
5553.jpg was skipped
9565.jpg was skipped
3491.jpg was skipped
7968.jpg was skipped
660.jpg was skipped
9171.jpg was skipped
10404.jpg was skipped
3300.jpg was skipped
4833.jpg was skipped
140.jpg was skipped
7978.jpg was skipped
Thumbs.db was skipped
11210.jpg was skipped
9778.jpg

In [5]:
print(training_data[1])

[array([[249, 249, 131, ..., 255, 254, 249],
       [250, 250, 137, ..., 254, 255, 251],
       [252, 250, 126, ..., 157, 154, 154],
       ...,
       [ 95,  90,  50, ..., 118, 141, 114],
       [ 87,  71,  56, ..., 100, 111, 108],
       [110,  68,  74, ..., 125, 101, 113]], dtype=uint8), 0]


In [6]:
#separate images and labels
x,y=[],[]

for features,label in training_data:
    x.append(features)
    y.append(label)
    
print('x and y datasets created')



x and y datasets created


In [9]:
numpy.array(x).shape

(25002, 70, 70)

In [ ]:
#cnn model will require an imput of 4 dimensions in the form of a numpy array
x=numpy.array(x)
print(x)
x=x.reshape(-1,img_size,img_size,1)
x=x/255 #normalize
y=numpy.array(y)
print('dataset shuffled and normalized')
print(x)

# Train the model

In [6]:
#import libraries
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D

In [7]:
model= Sequential()

#1st layer should always have input shape
# create a conv layer with 64 nodes and 3x3 window size
# convolution does not do any machine learning it only extracts the features 
model.add(Conv2D(64,(3,3), input_shape=x.shape[1:], activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3))) # maxpool with window size 3x3. Max pool will extract the most prominant features
 
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3))) 

model.add(Flatten()) #always flatten before a dense layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3)) # to prevent overfitting

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(2,activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x,y,epochs=3,validation_split=0.1) #validation split divides the data set into testing and traing data

model.save('catsdogs.model')

Epoch 1/3
704/704 [==============================] - 37s 9ms/step - loss: 0.6441 - accuracy: 0.6177 - val_loss: 0.5549 - val_accuracy: 0.7157
Epoch 2/3
704/704 [==============================] - 6s 8ms/step - loss: 0.5338 - accuracy: 0.7365 - val_loss: 0.5105 - val_accuracy: 0.7549
Epoch 3/3
704/704 [==============================] - 6s 8ms/step - loss: 0.4795 - accuracy: 0.7729 - val_loss: 0.4729 - val_accuracy: 0.7829
INFO:tensorflow:Assets written to: catsdogs.model/assets


In [8]:
from tensorflow.keras.models import load_model

In [9]:
img_array=cv2.imread('dog.jpg',cv2.IMREAD_GRAYSCALE)
img_array=cv2.resize(img_array,(70,70))
img_array=img_array.reshape(-1,70,70,1)

#load the model
#model=load_model('models/catsdogs.model')

prediction=model.predict([img_array])

print(categories[numpy.argmax(prediction)])

error: ignored